# IBM Ponder This - June 2024

## Problem Statement

$(A,B,C)$ is called a **Pythagorean triple** if $A$, $B$, $C$ are positive integers and $A^2+B^2=C^2$.

For example, $(3,4,5)$, $(24,7,25)$, $(20,21,29)$,  and $(1333,444,1405)$ are all Pythagorean triples.

We know that there's no triple such that $\frac{A}{B}=\pi$, because $\pi$ is irrational, but we can try to get close to that. The triple $(24,7,25)$ approximates $\pi$, but not very well, because $\frac{24}{7}\approx 3.4285$ , so $\left|\frac{24}{7}-\pi\right|<0.3$, but not $\left|\frac{24}{7}-\pi\right|<10^{-1}$.

Your goal Find a Pythagorean triple $(A,B,C)$ of numbers with 100 digits or less, such that $\left|\frac{A}{B}-\pi\right|<10^{-20}$.

Give your solution in the format
$A = 24$
$B = 7$
$C = 25$

A bonus "*" will be given for finding a Pythagorean triple $(D,E,F)$ of numbers with 100 digits or less, such that $\left|\frac{D}{E}-\pi\right|<10^{-95}$. Give your solution in the same format as above.

## Solution

Pythagorean triplets can be generated using Euclid's formula from two integers $m$ and $n$ as

\begin{align}
    A &= m^2 - n^2 \\
    B &= 2mn \\
    C &= m^2 + n^2.
\end{align}

Here we first generated the continued fraction of $\pi$. Then, we tried to generate pythagorean triplets that are close to the continued fraction approximation. First we generate the $m$ in Euclid's formula based on one randomly selected continued fraction of $\pi$. Assuming the continued fraction is given as $\frac{Y}{X}$, $m$ is given by

\begin{equation}
    m = \sqrt{\frac{X^2}{4Y - 1}}
\end{equation}

Then we add some random integer to $m$ to perturb it (i.e., $m$ becomes $m + \epsilon$). For the resulting $m$, we generate the corresponding $n$ that is the closest approximation to $\pi$ as

\begin{equation}
    n = m \left(\sqrt{\pi^2 + 1} - \pi \right)
\end{equation}

From $m$ and $n$, we generate the pythagorean triplet and compare $\frac{A}{B}$ to $\pi$ and count the matching decimals. Although the first question is answered directly, this does not allow to solve the bonus question in a reasonable amount of time. Note that the solution was to use the continued fraction method on $\sqrt{\pi^2 + 1} - \pi$ which we have computed to calculate $n$.

In [1]:
from mpmath import mp
import fractions
import sympy as sp
from fractions import Fraction
import random

# Set precision to 200 decimal places (more than needed for safety)
mp.dps = 200

# Value of pi to high precision
pi = mp.pi

# Function to compute continued fraction coefficients
def continued_fraction(x, n):
    a = []
    for _ in range(n):
        a.append(int(x))
        x = 1 / (x - int(x))
    return a

# Function to compute convergents from continued fraction coefficients
def convergents(a):
    convs = []
    for i in range(len(a)):
        convs.append(fractions.Fraction(a[i], 1))
        for j in range(i - 1, -1, -1):
            convs[-1] = a[j] + 1 / convs[-1]
    return convs

desired_precision = 10**(-110)

# Compute the continued fraction coefficients of pi
n = 200
cf = continued_fraction(pi, n)

# Generate convergents
convs = convergents(cf)


# Define the value of pi to a high precision
pi_value = sp.N(sp.pi, 110)  # Adjust the precision as needed

def count_matching_decimals(fraction, target_value, precision=110):
    # Convert the Fraction to a SymPy Rational
    rational_value = sp.Rational(fraction.numerator, fraction.denominator)

    # Convert the Rational to a high precision float
    fraction_value = sp.N(rational_value, precision)

    # Convert both values to strings
    fraction_str = str(fraction_value)
    target_str = str(target_value)

    # Compare the decimal places
    match_count = 0
    for f_char, t_char in zip(fraction_str, target_str):
        if f_char == t_char:
            match_count += 1
        else:
            break

    return match_count

fracs = []
# Check the accuracy of each Fraction object
for i, frac in enumerate(convs):
    match_count = count_matching_decimals(frac, pi_value)
    if len(str(sp.sqrt(frac.numerator**2 + frac.denominator**2).evalf(200)).split('.')[0]) > 100:
        break
    if match_count >= 95:
        fracs.append(frac)

curr_min = 10
curr_max = 0
prev_max = 16
has_break = False
for _ in range(100000000000):
    k = random.randint(1, len(fracs)-1)
    frac = fracs[k]
    A = sp.Rational(frac.numerator)
    B = sp.Rational(frac.denominator)
    m_ = sp.sqrt(B**2 / (4 * A - 1))
    m_ = sp.Integer(m_)
    for i in range(10000):
        i = random.randint(-10**50, 10**50)
        m = m_ + i
        if m < 0:
            m = abs(m)
        n = (m * (sp.sqrt(pi_value**2 + 1) - pi_value))
        n = sp.Integer(n)

        numerator = m**2 - n**2
        denominator = 2 * m * n
        res = sp.Rational(numerator, denominator).evalf(110)

        if abs(res - pi_value) < curr_min:
            curr_min = min(curr_min, abs(res - pi_value))
            curr_max = max(curr_max, count_matching_decimals(Fraction(m**2 - n**2, 2*m*n), pi_value))
            if curr_max > prev_max:
                print(curr_max)
                print((m, n, A, B))
                prev_max = curr_max
            if curr_max >= 97:
                print(m)
                print(n)
                has_break = True
                break
    if has_break:
        break

50
(97570731819923344020990123466117932241257165673755, 15154262207871029252010795087607194953900303004206, 694071435626132429113895797832419096687670413460608890005225360598011405945744302273417269266149443, 220929799677574407381056516026655867328746609968035614396202512410083914136283299148169604358477235)
51
(96084610618968917896378503549378184303015500930121, 14923444318818967197292690571716093813983616406973, 694071435626132429113895797832419096687670413460608890005225360598011405945744302273417269266149443, 220929799677574407381056516026655867328746609968035614396202512410083914136283299148169604358477235)
52
(79428550603307554660492259893313968223847091131541, 12336497433012909240447117204514275257229711848773, 694071435626132429113895797832419096687670413460608890005225360598011405945744302273417269266149443, 220929799677574407381056516026655867328746609968035614396202512410083914136283299148169604358477235)
53
(90721915313703959827271784762532716419943479696907, 14090533780165

KeyboardInterrupt: 